<a href="https://colab.research.google.com/github/cagutierrezgu/My_Portfolio/blob/main/SQL%20for%20mental%20health%20data%20analysis/Data_loading/Data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset and brief overview

A continuación se mostrarán las librerías a trabajar a lo largo del proyecto. Cabe aclarar que este notebook está pensado para ejecutarse bajo el ambiente de Google Colab, así que asume que librerías como Numpy, Pandas, Matplotlib, Seaborn, Plotly y Sqlite3 están ya descargadas.

Por otro lado, para la descarga de datos se usará la librería gdown, así que adelantes se presenta la descarga de la misma

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Ahora se importan las librerías ya mencionadas para el tratamiento de los datos a trabajar

In [ ]:
import gdown
import numpy as np
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Para la carga de datos, es posible dirigirse directamente a la página web de [Kaggle](https://www.kaggle.com/datasets/anth7310/mental-health-in-the-tech-industry), descargar la base de datos a usar y alojarla en el entorno de ejecución de este notebook. Otra alternativa es la que se mostrará en las siguientes celdas, donde se descargará el archivo comprimido ubicado en una carpeta de Google Drive de lectura pública.

Se define entonces una función que descargue un archivo de la red a partir de su url y lo guarde con un nombre dado con la librería gdown

In [ ]:
def dataload(link, name):
    gdown.download(link, name, quiet=False)

Posteriormente, se hace uso de la función definida teniendo la dirección del archivo y asignándole un nombre al archivo

In [ ]:
url = 'https://drive.google.com/uc?export=download&id=1oifXtw2gGg8YkiHN176K1enLCD50_9Zg'
name = 'archive.zip'
dataload(url,name)

Downloading...
From: https://drive.google.com/uc?export=download&id=1oifXtw2gGg8YkiHN176K1enLCD50_9Zg
To: /content/archive.zip
100%|██████████| 2.07M/2.07M [00:00<00:00, 28.8MB/s]


El archivo descargado es una compresión de la base de datos que se quiere utilizar, así que es necesario descomprimir el mismo

In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: mental_health.sqlite    


El siguiente paso consiste en realizar la conexión con la base de datos para trabajar con esta en el entorno de Python. Lo anterior es posible haciendo uso de la librería sqlite3 como se muestra en la siguiente celda

In [ ]:
connection = sqlite3.connect('mental_health.sqlite')

Una vez hecha esta conexión, es importante conocer la base de datos de trabajo, por ejemplo, es importante conocer el número de tablas que tiene dicho conjunto de información

In [ ]:
query = """
SELECT
  name 
FROM 
  sqlite_master 
WHERE 
  type ='table';
"""
df = pd.read_sql(sql=query, con=connection)
display(df)

,name
0,Answer
1,Question
2,Survey


Son 3 las tablas de información que tiene esta base de datos, las cuales se visualizarán más adelante. En un orden comprensible, la primera de estas tablas es la de nombre Survey, a la cual se accede como sigue

In [ ]:
query = """
SELECT
  *
FROM
  Survey;
"""
answers = pd.read_sql(sql=query, con=connection)
display(answers)

,SurveyID,Description
0,2014,mental health survey for 2014
1,2016,mental health survey for 2016
2,2017,mental health survey for 2017
3,2018,mental health survey for 2018
4,2019,mental health survey for 2019


La anterior tabla hace referencia a los años de realización de las encuestas relacionadas con salud mental a personas empleadas en la industria Tech. Veamos ahora que las demás tablas proporcionan mayor información que la ya vista, por ejemplo, la tabla 'Question'

In [ ]:
query = """
SELECT
  *
FROM
  Question;
"""
questions = pd.read_sql(sql=query, con=connection)
display(questions)

,questiontext,questionid
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5
...,...,...
100,Do you think that team members/co-workers woul...,114
101,"If yes, what condition(s) have you been diagno...",115
102,"If maybe, what condition(s) do you believe you...",116
103,Which of the following best describes your wor...,117


Como su nombre lo indica, esta tabla deja ver la lista de preguntas realizadas en las versiones de las encuestas a estudiar; además, cada una de estas preguntas tiene asociada un ID que las caracteriza. Por último, existe una tabla dedicada a las respuestas de los encuestados titulada como 'Answer'

In [ ]:
query = """
SELECT
  *
FROM
  Answer;
"""

surveys = pd.read_sql(sql=query, con=connection)
display(surveys)

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1
...,...,...,...,...
236893,Other,2016,2689,117
236894,Support,2016,2690,117
236895,Back-end Developer,2016,2691,117
236896,DevOps/SysAdmin,2016,2692,117


Como se observa en el anterior dataframe comparado con los ya vistos, este es el que posee mayor información. Una de sus columnas guarda las respuestas de cada encuestado, mientras que la columna 'QuestionID' indica la etiqueta de la pregunta que está respondiendo. Las 2 columnas restantes presentan el año respectivo de dicha respuesta, y la columna 'UserID' muestra una etiqueta para la persona que dado una respuesta a una pregunta determinada.